# Projet Files d'Attentes

Cyril Nederveen et Mathis Brichet

## Modélisation

On considère un système de vélos partagés, type Vélib, où les vélos sont disponibles dans des stations sont empruntés pour faire des trajets uniquement d’une station à une autre. La capacité des stations est supposée illimitée pour l’accueil (on peut toujours rendre le vélo).

Ces hypothèses nous permettent de modéliser ce problème par un modèle de colonies. On considère une colonie par station et une colonie par trajet possible entre les stations. Notons $V$ le nombre de vélos considérés pour notre système et $S$ le nombre de stations considérées.

On note alors, pour $i,j \in \{1, ..., S\}^2, i \ne j$ :
 - $n_{ii}$ le nombre de vélos en attente dans la station $i$
 - $\mu_{ii}$ le taux de départ des vélos de la station $i$ (paramètre de la loi exponentielle régissant la demande) 
 - $n_{ij}$ le nombre de vélos en trajet entre les stations $i$ et $j$
 - $\mu_{ij}$ le temps de trajet moyen entre la station $i$ et la station $j$ (paramètre de la loi exponentielle régissant la durée)
 - $P_{ij}$ la probabilité qu'un utilisateur prenant un vélo à la station $i$ le rende à la station $j$

In [9]:
import numpy as np

# Table des temps moyens de trajets (convertis en heures) (mu_ij pour i != j)
Tps_trajets = 1/60 * np.array([[0, 3, 5, 7, 7],
                               [2, 0, 2, 5, 5],
                               [4, 2, 0, 3, 3], 
                               [8, 6, 4, 0, 2], 
                               [7, 7, 5, 2, 0]])

# Table des taux de départs par heures (mu_ii)
Taux_dep = [2.8, 3.7, 5.5, 3.5, 4.6]

# Matrice de routage
P = np.array([[0, 0.22, 0.32, 0.2, 0.25],
              [0.17, 0, 0.34, 0.21, 0.28],
              [0.19, 0.26, 0, 0.24, 0.31],
              [0.17, 0.22, 0.33, 0, 0.28],
              [0.18, 0.24, 0.35, 0.23, 0]])

Cela nous permet alors de définir $n = (n_{11}, n_{12}, ..., n_{SS})$ comme l'état de notre système.
L'espace d'état peut alors se noter $E_{S,V} = \{n \in \mathbb{N}^{SxS} \: tq. \: \sum_{ij} n_{ij} = V \}$, dont le cardinal est de l'ordre de $S^2V$.

Définissons $T_{ij,kl}$ l'opérateur tel que $T_{ij,kl}n = n - e_{ij} + e_{kl}$, avec $e_{ij}$ le vecteur de la base canonique de $\mathbb{N}^{SxS}$ d'indice $ij$. Nous notons alors $q(n, T_{ij,kl}n)$ le taux de transition entre $n$ et $T_{ij,kl}n$.  

Seules les transitions du type $ij \rightarrow jj$ ou $ii \rightarrow ij$ sont possibles donc toutes les autres ont des taux égale à 0. De plus, dans chaque colonie, chaque memebre de la colonie possède sa propre "horloge exponentielle", ce qui nous ammène à la formule suivante pour le taux de transition : 

$\forall i, j \in \{1, ..., S\}^2, i \ne j$
 - $q(n, T_{ij,jj}n) = \mu_{ij}n_{ij} = \lambda_{ij,jj}\phi_{ij}(n_{ij})$ pour $\mu_{ij} = \lambda_{ij,jj}$ et $n_{ij} = \phi_{ij}(n_{ij})$ 
 - $q(n, T_{ii,ij}n) = P_{ij}\mu_{ii}n_{ii} = \lambda_{ii,ij}\phi_{ii}(n_{ii})$ pour $P_{ij}\mu_{ii} = \lambda_{ii,ij}$ et $n_{ii} = \phi_{ii}(n_{ii})$ 

In [10]:
# Matrice des taux de type arrivée (1/Tps_trajets_ij)
Ta = 1/Tps_trajets
np.fill_diagonal(Ta, 0)
print(Ta)

# Matrice des taux de type départ (P_ij * Taux de départ_i)
Td = np.multiply(Taux_dep, P.T).T
print(Td)

[[ 0.         20.         12.          8.57142857  8.57142857]
 [30.          0.         30.         12.         12.        ]
 [15.         30.          0.         20.         20.        ]
 [ 7.5        10.         15.          0.         30.        ]
 [ 8.57142857  8.57142857 12.         30.          0.        ]]
[[0.    0.616 0.896 0.56  0.7  ]
 [0.629 0.    1.258 0.777 1.036]
 [1.045 1.43  0.    1.32  1.705]
 [0.595 0.77  1.155 0.    0.98 ]
 [0.828 1.104 1.61  1.058 0.   ]]


<ipython-input-10-593d01562d5d>:2: RuntimeWarning: divide by zero encountered in true_divide
  Ta = 1/Tps_trajets


Cette modélisation en colonies possède une probabilité stationnaire de la forme suivante :

$\pi(n) = \frac{1}{G} \prod_{i \ne j} \frac{\alpha_{ij}^{n_{ij}}}{n_{ij}!}$


### Question 1

Les équations de traffic nous permettent d'obtenir les relations entre les $\lambda_{ij,kl}$ et les $\alpha_{ij}$ de la probabilité stationnaire. celles-ci sont de la forme suivante pour notre modèle :

$\forall i, j \in \{1, ..., S\}^2$, $\alpha_{ij} \sum_{kl} \lambda_{ij,kl} = \sum_{kl} \alpha_{kl} \lambda_{kl,ij}$ 

Ces équations se simplifient en :
 - si $i = j$, $\alpha_{ii} \sum_{l} \lambda_{ii,il} = \sum_{k} \alpha_{ki} \lambda_{ki,ii}$ soit $\alpha_{ii} \sum_{l} P_{il} \mu_{ii} = \sum_{k} \alpha_{ki} \mu_{ki}$
 - si $i \ne j$, $\alpha_{ij} \lambda_{ij,jj} = \alpha_{ii} \lambda_{ii,ij}$ soit $\alpha_{ij} \mu_{ij} = \alpha_{ii} P_{ij} \mu_{ii}$

On remarque bien ici les égalités du type "ce qui arrive" $=$ "ce qui part"

### Question 2

Pour 1 vélo, le cardinal de notre espace d'état est de $S^2 = 25$

### Question 3

Pour résoudre numériquement le système d’équations donnant les $\alpha_{ij}$, on met le système sous forme de calcul matriciel $M\alpha = X$ où $\alpha$ est le vecteur colonne des $\alpha_{ij}$, où l'on à remplacer une ligne de M par des 1 (S^2+1 équations pour S^2 inconnues si on ajoute la condition de normalisation) et où X est le vecteur $(1,0,...,0)$. On a alors α = M−1X.

Pour se faire, on a "applati" la matrice des $\alpha_{ij}$ en un vecteur ou la coordonnées $(i-1)S+j$ coorespond au coefficient $(i,j)$ de la matrice d'origine. L'équation qui a été remplacé par celle de normalisation est celle d'indice $i = j = S$. 

In [24]:
# Résolution du système M * alpha = X

S, V = 5, 1
X = np.zeros(5*5)
X[0] = 1
print(X)

M = np.zeros((5*5, 5*5))
M[0,:] = np.ones(5*5)
for i in range(S):
    for j in range(S):
        if i == j:
            # On enlève la dernière équation
            if i == S-1:
                continue
            # La somme des alpha_ki du coté droit
            for k in range(S):
                M[i*S + j + 1, k*S + i] = -Ta[k,i]
            # Le alpha_ii de l'équation 1+i+j (la première équation étant celle de normalisation)
            M[i*S + j + 1, i*S + i] = np.sum(Td[i,:])
        else:
            # Coté gauche (alpha_ij)
            M[i*S + j + 1, i*S + j] = Ta[i,j]
            # Coté droit (alpha_ii)
            M[i*S + j + 1, i*S + i] = -Td[i,j]
        # print(f"i = {i}, j = {j} :\n {M[i+j+1,:]}\n")

# print(M.size)
print(M)

alpha = np.linalg.solve(M, X)
print(alpha)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
i = 0, j = 0 :
 [  2.772        0.           0.           0.           0.
 -30.           0.           0.           0.           0.
 -15.           0.           0.           0.           0.
  -7.5          0.           0.           0.           0.
  -8.57142857   0.           0.           0.           0.        ]

i = 0, j = 1 :
 [-0.616 20.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.   ]

i = 0, j = 2 :
 [-0.896  0.    12.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.   ]

i = 0, j = 3 :
 [-0.56        0.          0.          8.57142857  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0

Une fois les $\alpha$ calculés, il reste à calculer la constante de normalisation $G$. Pour cela, on crée tous les états possible de système afin de sommer dessus.

In [25]:
# Calcul de la probabilité stationnaire

G = 0
for i in range(S):
    for j in range(S):
        # Etat ou n_ij = 1 
        G += alpha[i*S + j]

# Probabilité que n_ij = 1
def pi(i,j):
    return 1 - (1/G) * alpha[i*S + j]

proba_station_vide = [pi(i,i) for i in range(S)]
print(proba_station_vide)

[0.8307407684007514, 0.8388769024451824, 0.8580340094114649, 0.8385488196253973, 0.8509393126289504]


## Simulateur de notre système

In [ ]:
# Code du simulateur

Ntest = np.full((nb_stations,nb_stations), int(nb_velos/(nb_stations**2)))

def transition(N):
    #
    A_arrivee = np.multiply(Ta, N).flatten()
    A_depart = (np.multiply(np.diag(N), Td.T).T).flatten()

    A_tot = np.concatenate([A_arrivee, A_depart])

    r = np.cumsum(A_tot)/np.sum(A_tot)

    u = 0.98 #rand()
    w = 0

    while u > r[w]:
        w+=1

    print(u)
    print(r)
    print(w)
    # Transition de type arrivée
    if w < nb_stations**2 :
        return "Arrivée", w//nb_stations, w%nb_stations
    
    # Transition de type départ
    else : 
        w -= nb_stations**2
        return "Départ", w//nb_stations, w%nb_stations
    
print(transition(Ntest))
# Paramètre de la loi expo du temps de séjour en un état n
def A(N):
    A_N = 0
    for i in range(nb_stations):
        for j in range(i+1, nb_stations):
            A_N += N[i,j]/Tps_trajets[i,j] + N[j,i]/Tps_trajets[j,i] + P[i,j] * N[i,i]*P[i,j]*Taux_dep[i] + N[j,j]*P[j,i]*Taux_dep[j]
    return A_N

In [ ]:
# état initial (réparti uniformément sur les colonnies)
N = np.full((nb_stations,nb_stations), int(nb_velos/(nb_stations**2)))

#N = randint(0,int(nb_velos/(nb_stations**2)) + 1,(nb_stations, nb_stations))
#N[nb_stations-1, nb_stations-1] = nb_stations - sum(N) - N[nb_stations-1, nb_stations-1] 

print(A(N))

In [ ]:
horizon=20
temps=0

while temps <= horizon :
    duree = -1/A(N) * log(rand())
    # prochaine transition ?
    



In [ ]:
rho=0.5
horizon=10000
temps=0
N=0
K=int(np.trunc(log(0.001)/log(rho)))+1
proba_stat=np.zeros(K+1)
nb_moyen_clients=0

def transitions(N): 
    if (N==0):
        return([rho,[1],[1]]) 
    else:
        return([rho+1,[N-1,N+1],[1/(rho+1),rho/(rho+1)]])

while(temps<= horizon): a = transitions(N)
#
#
print N,a
duree=np.random.exponential(1/a[0])
temps += duree
proba_stat[min(N,K)]+= duree
N=np.random.choice(a[1],p=a[2])
print N
proba_stat =proba_stat/horizon
pylab.clf()
pylab.bar(np.arange(K+1),proba_stat)
pylab.plot(np.arange(K+1),(1-rho)*rho**np.arange(K+1),color='red')
pylab.savefig('geometric')

## Simulation pour 1 vélo

In [ ]:
# Simulation pour 1 vélo

### Question 4

Nous remarquons ici que notre simulation nous permet d'obtenir une approximation précise de la probabilité stationnaire.

## Simulation pour 50 vélos

### Question 5 

Dans ce cas, le cardinal de l'espace d'état est de l'ordre de $S^2V = 1250$. Devant un nombre d'états aussi important, la simulation semble être la seule solution envisageable pour calculer ces probabilités stationnaires.

In [5]:
%matplotlib inline 
import scipy
from pylab import* 
import pylab as pylab

nb_stations = 5
nb_velos = 50


In [66]:
Ntest = np.full((nb_stations,nb_stations), int(nb_velos/(nb_stations**2)))

def transition(N):
    #
    A_arrivee = np.multiply(Ta, N).flatten()
    A_depart = (np.multiply(np.diag(N), Td.T).T).flatten()

    A_tot = np.concatenate([A_arrivee, A_depart])

    r = np.cumsum(A_tot)/np.sum(A_tot)

    u = 0.98 #rand()
    w = 0

    while u > r[w]:
        w+=1

    print(u)
    print(r)
    print(w)
    # Transition de type arrivée
    if w < nb_stations**2 :
        return "Arrivée", w//nb_stations, w%nb_stations
    
    # Transition de type départ
    else : 
        w -= nb_stations**2
        return "Départ", w//nb_stations, w%nb_stations
    
print(transition(Ntest))
# Paramètre de la loi expo du temps de séjour en un état n
def A(N):
    A_N = 0
    for i in range(nb_stations):
        for j in range(i+1, nb_stations):
            A_N += N[i,j]/Tps_trajets[i,j] + N[j,i]/Tps_trajets[j,i] + P[i,j] * N[i,i]*P[i,j]*Taux_dep[i] + N[j,j]*P[j,i]*Taux_dep[j]
    return A_N

0.98
[0.         0.05557752 0.08892404 0.11274297 0.13656191 0.21992819
 0.21992819 0.30329448 0.33664099 0.3699875  0.41167064 0.49503693
 0.49503693 0.55061445 0.60619197 0.62703354 0.6548223  0.69650544
 0.69650544 0.77987173 0.80369067 0.8275096  0.86085612 0.9442224
 0.9442224  0.9442224  0.94593419 0.94842406 0.94998023 0.95192544
 0.95367336 0.95367336 0.95716918 0.95932837 0.96220729 0.96511121
 0.969085   0.969085   0.97275312 0.9774911  0.97914453 0.98128427
 0.98449387 0.98449387 0.98721717 0.98951808 0.99258596 0.99705995
 1.         1.        ]
41
('Départ', 3, 1)


In [39]:
# état initial (réparti uniformément sur les colonnies)
N = np.full((nb_stations,nb_stations), int(nb_velos/(nb_stations**2)))

#N = randint(0,int(nb_velos/(nb_stations**2)) + 1,(nb_stations, nb_stations))
#N[nb_stations-1, nb_stations-1] = nb_stations - sum(N) - N[nb_stations-1, nb_stations-1] 

print(A(N))

705.4393485714286


In [ ]:
horizon=20
temps=0

while temps <= horizon :
    duree = -1/A(N) * log(rand())
    # prochaine transition ?
    



In [13]:
rho=0.5
horizon=10000
temps=0
N=0
K=int(np.trunc(log(0.001)/log(rho)))+1
proba_stat=np.zeros(K+1)
nb_moyen_clients=0

def transitions(N): 
    if (N==0):
        return([rho,[1],[1]]) 
    else:
        return([rho+1,[N-1,N+1],[1/(rho+1),rho/(rho+1)]])

while(temps<= horizon): a = transitions(N)
#
#
print N,a
duree=np.random.exponential(1/a[0])
temps += duree
proba_stat[min(N,K)]+= duree
N=np.random.choice(a[1],p=a[2])
print N
proba_stat =proba_stat/horizon
pylab.clf()
pylab.bar(np.arange(K+1),proba_stat)
pylab.plot(np.arange(K+1),(1-rho)*rho**np.arange(K+1),color='red')
pylab.savefig('geometric')

0.5413503559189221
